# Evaluación I Módulo 2 (sprint 2)
---

In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import mysql.connector
import sys

sys.path.append('../')
from src import soporte_variables as spv
from src import soporte_funciones as spf
from src import soporte_clases as spc

En esta evaluación nos enfrentamos a un desafío emocionante de trabajar en un proyecto real para una empresa que realiza un estudio de universidades por el mundo. El proyecto tiene como objetivo identificar todas las universidades ubicadas en tres países específicos: Estados Unidos, Canadá y Argentina.  

Para llevar a cabo esta tarea, utilizaremos la API de "Universities Hipolabs", una fuente confiable y completa de información sobre las universidades en todo el mundo. Con la ayuda de esta API, podemos acceder a una gran cantidad de datos relevantes, incluyendo el nombre de la universidad, la ciudad donde esta ubicada, el nombre de la institución y otra información importante que nos permitirá llevar a cabo un análisis detallado.  

Es importante tener en cuenta que este proyecto requerirá un conocimiento profundo de herramientas y técnicas de análisis de datos, así como habilidades en programación y manejo de APIs. También es importante tener una comprensión sólida de la estructura y organización de los datos, ya que esto nos permitirá hacer preguntas importantes y obtener respuestas significativas a partir de los datos.  

En resumen, esta prueba técnica ofrece una excelente oportunidad para demostrar habilidades y conocimientos en análisis de datos y programación, mientras se trabaja en un proyecto real y relevante para una empresa. Al finalizar del proyecto, esperamos obtener información valiosa que ayudará a la empresa a tomar decisiones más informadas sobre las universidades en los tres países objetivo.  

1. Utilizando la API extraed toda la información que podáis de ella. La url para hacer las llamadas es:
    ```python
    API_URL = "http://universities.hipolabs.com/search?country=NOMBREPAIS"
    ```

In [2]:
# los paises que nos interesan son:
print(f'Vamos a realizar la extracción para los siguientes paises: {spv.paises}')

Vamos a realizar la extracción para los siguientes paises: ['Argentina', 'Canada', 'United States']


In [3]:
# utilizamos una función para obtener los datos de Argentina a modo de ejemplo para explorar los resultados que tendremos para cada país
argentina = spf.extraccion_json(spv.paises[0])
print('Printeamos los dos primeros resultados descargados de la API a modo de ejemplo:')
argentina[0:2] # printeamos una muestra

Status code extracción Argentina: 200
Reason extracción Argentina: OK
El resultado obtenido es una lista de diccionarios. Nº de elementos de la lista resultado: 87
Las keys de cada diccionario son 6: dict_keys(['state-province', 'domains', 'name', 'country', 'web_pages', 'alpha_two_code'])
Printeamos los dos primeros resultados descargados de la API a modo de ejemplo:


[{'state-province': 'Buenos Aires',
  'domains': ['atlantida.edu.ar'],
  'name': 'Universidad Atlantida Argentina',
  'country': 'Argentina',
  'web_pages': ['http://www.atlantida.edu.ar/'],
  'alpha_two_code': 'AR'},
 {'state-province': 'Buenos Aires',
  'domains': ['austral.edu.ar'],
  'name': 'Universidad Austral Buenos Aires',
  'country': 'Argentina',
  'web_pages': ['http://www.austral.edu.ar/'],
  'alpha_two_code': 'AR'}]

In [4]:
# recordamos que teníamos la siguiente variable con los paises a analizar:
spv.paises

['Argentina', 'Canada', 'United States']

In [5]:
# En este caso son solo tres paises pero por si fueran más podríamos automatizar la obtención de los datos, una vez conocida su estructura de la siguiente manera:
df = pd.DataFrame()
for pais in spv.paises:
    datos = spf.extraccion_json(pais)
    print(f'Nº de elementos de la lista resultado de {pais}: {len(datos)}') # obtenemos el número de elementos
    print(f'Las keys de cada diccionario de {pais} son {len(datos[0])}: {datos[0].keys()}')
    df_pais = pd.DataFrame(datos)
    print(f'Añadimos al dataframe resultado las {df_pais.shape[0]} filas y {df_pais.shape[1]} columnas de {pais}')
    df = pd.concat([df, df_pais], axis=0, ignore_index=True) # concatenamos los dataframes, ignorando el índice
    print('----------------------------------------')
print(f'El dataframe resultado tiene {df.shape[0]} filas y {df.shape[1]} columnas')

Status code extracción Argentina: 200
Reason extracción Argentina: OK
El resultado obtenido es una lista de diccionarios. Nº de elementos de la lista resultado: 87
Las keys de cada diccionario son 6: dict_keys(['state-province', 'domains', 'name', 'country', 'web_pages', 'alpha_two_code'])
Nº de elementos de la lista resultado de Argentina: 87
Las keys de cada diccionario de Argentina son 6: dict_keys(['state-province', 'domains', 'name', 'country', 'web_pages', 'alpha_two_code'])
Añadimos al dataframe resultado las 87 filas y 6 columnas de Argentina
----------------------------------------
Status code extracción Canada: 200
Reason extracción Canada: OK
El resultado obtenido es una lista de diccionarios. Nº de elementos de la lista resultado: 154
Las keys de cada diccionario son 6: dict_keys(['state-province', 'domains', 'name', 'country', 'web_pages', 'alpha_two_code'])
Nº de elementos de la lista resultado de Canada: 154
Las keys de cada diccionario de Canada son 6: dict_keys(['state

In [6]:
# guardamos el dataframe en .csv y .pkl
df.to_csv('../datos/universidades_arg-can_usa.csv')
df.to_pickle('../datos/universidades_arg-can_usa.pkl')

In [7]:
# Antes de continuar realizamos una pequeña exploración del dataframe con la siguiente función
spf.explorar_df(df, nombre = 'Argentina-Canada-USA')

EXPLORACIÓN DEL DATAFRAME ARGENTINA-CANADA-USA
---------------------------------------------------------------------------
Las primeras 5 filas del dataframe Argentina-Canada-USA son:


,state-province,domains,name,country,web_pages,alpha_two_code
0,Buenos Aires,[atlantida.edu.ar],Universidad Atlantida Argentina,Argentina,[http://www.atlantida.edu.ar/],AR
1,Buenos Aires,[austral.edu.ar],Universidad Austral Buenos Aires,Argentina,[http://www.austral.edu.ar/],AR
2,Ciudad Autónoma de Buenos Aires,[caece.edu.ar],"Universidad CAECE, Buenos Aires",Argentina,[http://www.caece.edu.ar/],AR
3,Ciudad Autónoma de Buenos Aires,[cema.edu.ar],Instituto Universitario CEMA,Argentina,[http://www.cema.edu.ar/],AR
4,Ciudad Autónoma de Buenos Aires,[iese.edu.ar],Instituto de Enseñanza Superior del Ejército,Argentina,[http://www.iese.edu.ar/],AR


---------------------------------------------------------------------------
Las últimas 5 filas del dataframe Argentina-Canada-USA son:


,state-province,domains,name,country,web_pages,alpha_two_code
2517,None,[vul.edu],Virginia University of Lynchburg,United States,[https://www.vul.edu/],US
2518,None,[voorhees.edu],Voorhees University,United States,[https://www.voorhees.edu/],US
2519,None,[wvstate.edu],West Virginia State University,United States,[https://www.wvstateu.edu/],US
2520,None,[wileyc.edu],Wiley College,United States,[https://www.wileyc.edu/],US
2521,None,[wssu.edu],Winston-Salem State University,United States,[https://www.wssu.edu/],US


---------------------------------------------------------------------------
A comntinuación se muestran 10 filas aleatorias del dataframe Argentina-Canada-USA:


,state-province,domains,name,country,web_pages,alpha_two_code
1322,None,[uor.edu],University of Redlands,United States,[http://www.uor.edu/],US
1214,None,[ucr.edu],"University of California, Riverside",United States,[http://www.ucr.edu/],US
2005,None,[tmcc.edu],Truckee Meadows Community College,United States,[http://www.tmcc.edu],US
1092,None,[sckans.edu],Southwestern College (KS),United States,[http://www.sckans.edu/],US
1054,None,[slc.edu],Sarah Lawrence College,United States,[http://www.slc.edu/],US
2118,None,[johnstoncc.edu],Johnston Community College,United States,[http://www.johnstoncc.edu],US
429,None,[bentley.edu],Bentley College,United States,[http://www.bentley.edu/],US
2279,None,[fpctx.edu],Frank Phillips College,United States,[http://www.fpctx.edu],US
1349,None,[utexas.edu],University of Texas at Austin,United States,[http://www.utexas.edu/],US
2491,None,[livingstone.edu],Livingstone College,United States,[http://livingstone.edu/],US


---------------------------------------------------------------------------
El dataframe Argentina-Canada-USA tiene 2522 filas y 6 columnas
---------------------------------------------------------------------------
A continuación el resultado del método .info() incluyendo los tipos de dato de cada columna:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2522 entries, 0 to 2521
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   state-province  273 non-null    object
 1   domains         2522 non-null   object
 2   name            2522 non-null   object
 3   country         2522 non-null   object
 4   web_pages       2522 non-null   object
 5   alpha_two_code  2522 non-null   object
dtypes: object(6)
memory usage: 118.3+ KB
---------------------------------------------------------------------------
El número de nulos por columna en valor absoluto y porcentaje es:
state-province: nº de nulos: 2249. % de nulos: 89

,count,unique,top,freq
state-province,273,49,Ontario,46
domains,2522,2491,[ccc.edu],7
name,2522,2507,Southeastern Community College,2
country,2522,3,United States,2281
web_pages,2522,2519,[http://www.coastalpines.edu/],2
alpha_two_code,2522,3,US,2281


---------------------------------------------------------------------------
El dataframe Argentina-Canada-USA tiene las siguientes columnas: 
Index(['state-province', 'domains', 'name', 'country', 'web_pages',
       'alpha_two_code'],
      dtype='object')
---------------------------------------------------------------------------
El numero de valores distintos de cada columna es:
state-province: 49
domains: 2491
name: 2507
country: 3
Los valores únicos de la columna "country" son: ['Argentina' 'Canada' 'United States']
web_pages: 2519
alpha_two_code: 3
Los valores únicos de la columna "alpha_two_code" son: ['AR' 'CA' 'US']


2. Una vez tengáis todos los datos de la API, deberéis realizar una serie de procesos de limpieza, estos incluyen:  
    - Cambiad los nombres de las columnas para homogeneizarlas, tenemos columnas que tienen - y otras _. Unifícalo para que todo vaya con _.   
    - La columna de domains nos da una información similar a la de web_pages. Eliminad la columna domains.  

In [8]:
# comprobamos que el índice es continuo
df.index

RangeIndex(start=0, stop=2522, step=1)

In [9]:
# antes de continuar realizamos una copia del dataframe original para realizar la limpieza sobre él
df2 = df.copy()
df2.sample()

,state-province,domains,name,country,web_pages,alpha_two_code
1239,None,[uga.edu],University of Georgia,United States,[http://www.uga.edu/],US


In [10]:
# realizamos la homogeneización de columnas. Además de reemplazar los '-' por '_' utilizamos un .strip() por si hubiera algún espacio al inicio o el final que no vieramos
col_new = {col : col.strip().replace('-', '_') for col in df.columns} 
df2 = df2.rename(columns = col_new)
df2.columns

Index(['state_province', 'domains', 'name', 'country', 'web_pages',
       'alpha_two_code'],
      dtype='object')

In [11]:
df2 = df2.reindex(columns=[df2.columns[3], df2.columns[0], df2.columns[4], df2.columns[2], df2.columns[1], df2.columns[-1]]) # ordenamos las columnas 
df2.sample()

,country,state_province,web_pages,name,domains,alpha_two_code
1046,United States,None,[http://www.salisbury.edu/],Salisbury University,[salisbury.edu],US


In [12]:
# eliminamos la columna con información duplicada
df2.drop(columns='domains', inplace=True)
df2.sample()

,country,state_province,web_pages,name,alpha_two_code
2514,United States,None,[https://www.lincoln.edu/],The Lincoln University,US


3. Si exploramos la columna de web_pages, nos daremos cuenta que hay universidades, como por ejemplo la Universidad de "Cégep de Saint-Jérôme" de Canadá que en su columna de web_pages tiene más de un valor dentro de la lista. Esto es poco práctico y puede llegar a no tener sentido. el objetivo de este ejericio es que usando el método explode de pandas separéis cada elemento de la lista en una fila nueva. [Aquí](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.explode.html) tenéis la documentación de este método.

In [13]:
# comprobamos lo que nos indica el enunciado
for indice, row in df2.iterrows():
    if len(row['web_pages']) > 1:
        print(f'{row["name"]} ----> {row["web_pages"]}')


Cégep de Saint-Jérôme ----> ['https://www.cstj.qc.ca', 'https://ccmt.cstj.qc.ca', 'https://ccml.cstj.qc.ca']
Langara College ----> ['http://www.langara.bc.ca/', 'https://langara.ca/']
St. Mary's University ----> ['http://www.stmarys.ca/', 'https://smu.ca/']
Augusta University ----> ['http://www.augusta.edu/', 'http://www.gru.edu/']
California State University, Fresno ----> ['http://www.csufresno.edu/', 'http://www.fresnostate.edu/']
Capella University ----> ['http://www.capella.edu/', 'http://www.capellauniversity.edu/']
Colorado Technical University ----> ['http://www.coloradotech.edu/', 'http://www.ctuonline.edu/']
Thomas Edison State University ----> ['http://www.tesc.edu/', 'http://www.tesu.edu/']
United States Air Force Academy ----> ['http://www.usafa.af.mil/', 'http://www.usafa.edu/']
University of Texas Southwestern Medical Center ----> ['http://www.swmed.edu/', 'http://www.utsouthwestern.edu/']
University of the Incarnate Word ----> ['http://www.uiw.edu/', 'http://www.uiwtx.ed

In [14]:
# separamos con el método .explode()
df2 = df2.explode('web_pages')
print(f'El nuevo dataframe tiene {df2.shape[0]} filas y {df2.shape[1]} columnas')
df2.sample()

El nuevo dataframe tiene 2535 filas y 5 columnas


,country,state_province,web_pages,name,alpha_two_code
717,United States,None,http://www.howardcc.edu/,Howard Community College,US


In [15]:
# nos fijamos en que tenemos un index no continuo
df2.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            2512, 2513, 2514, 2515, 2516, 2517, 2518, 2519, 2520, 2521],
           dtype='int64', length=2535)

In [16]:
# reseteamos el índice
df2.reset_index(drop=True, inplace=True)
df2.sample()

,country,state_province,web_pages,name,alpha_two_code
1418,United States,None,http://www.vsu.edu/,Virginia State University,US


In [17]:
# ahora sí es contínuo
df2.index

RangeIndex(start=0, stop=2535, step=1)

4. Una vez hayáis realizado el explode, chequead si tenéis duplicados basándonos unicamente en el nombre de la universidad, en caso de que si, eliminandlos.

In [18]:
# chequeamos que tenemos 28 duplicados si nos basamos en la columna 'name'
print(f'Si filtramos por la columna "name" tenemos {df2["name"].duplicated().sum()} duplicados')

Si filtramos por la columna "name" tenemos 28 duplicados


In [19]:
# antes de borrar los duplicados podríamos extraerlos por si nos interesase guardarlos y no perder los datos de las webs que vamos a eliminar del dataframe limpio (nos quedamos solo con la primera)
df_duplicados = df2[df2['name'].duplicated()]
print(f'El dataframe de los duplicados a eliminar tiene {df_duplicados.shape[0]} filas y {df_duplicados.shape[1]} columnas y se muestra a continuación:')
df_duplicados

El dataframe de los duplicados a eliminar tiene 28 filas y 5 columnas y se muestra a continuación:


,country,state_province,web_pages,name,alpha_two_code
88,Canada,Quebec,https://ccmt.cstj.qc.ca,Cégep de Saint-Jérôme,CA
89,Canada,Quebec,https://ccml.cstj.qc.ca,Cégep de Saint-Jérôme,CA
138,Canada,British Columbia,https://langara.ca/,Langara College,CA
179,Canada,None,https://smu.ca/,St. Mary's University,CA
410,United States,None,http://www.gru.edu/,Augusta University,US
478,United States,None,http://www.fresnostate.edu/,"California State University, Fresno",US
497,United States,None,http://www.capellauniversity.edu/,Capella University,US
563,United States,None,http://www.ctuonline.edu/,Colorado Technical University,US
601,United States,None,http://www.devry.edu/,DeVry Institute of Technology,US
777,United States,None,http://www.kings.edu/,King's College,US


In [20]:
# eliminamos los duplicados quedándonos con el primero en cada caso
df2.drop_duplicates(subset=['name'], inplace=True, ignore_index=True)
print(f'El nuevo dataframe tiene {df2.shape[0]} filas y {df2.shape[1]} columnas')
print('Comprobamos que el indice se ha reseteado:')
df2.index

El nuevo dataframe tiene 2507 filas y 5 columnas
Comprobamos que el indice se ha reseteado:


RangeIndex(start=0, stop=2507, step=1)

5. Si exploramos la columna de state_province veremos que hay universidades cuyo valor para esta columna es None. Cread una función para reemplazar los None por nulos de numpy.

In [21]:
# comprobamos que como dice el enunciado tenemos None
df2['state_province'].unique()

array(['Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', None,
       'Santiago Del Estero', 'Misiones', 'Catamarca', 'Formosa', 'Jujuy',
       'La Rioja', 'La Pampa', 'San Juan', 'San Luis', 'Tucumán',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Pennsylvania', 'NV', 'Iowa', 'VA', 'TX', 'Colorado', 'IN', 'CA',
       'South Carolina', 'Washington', 'NY', 'Texas', 'ND', 'MI', 'Ohio',
       'Florida', 'California', 'North Carolina', 'Michigan', 'GA',
       'New York, NY'], dtype=object)

In [22]:
# utilizamos el método .fillna para sustituir None por np.nan
df2['state_province'].fillna(value=np.nan,  axis=None, inplace=True)
df2['state_province'].unique()

array(['Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', nan,
       'Santiago Del Estero', 'Misiones', 'Catamarca', 'Formosa', 'Jujuy',
       'La Rioja', 'La Pampa', 'San Juan', 'San Luis', 'Tucumán',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Pennsylvania', 'NV', 'Iowa', 'VA', 'TX', 'Colorado', 'IN', 'CA',
       'South Carolina', 'Washington', 'NY', 'Texas', 'ND', 'MI', 'Ohio',
       'Florida', 'California', 'North Carolina', 'Michigan', 'GA',
       'New York, NY'], dtype=object)

6. Después del último cambio, os habréis dado cuenta que tenemos muchos valores nulos dentro de la columna de state_province, por lo que nuestro jefe nos pide que reemplacemos esos nulos por "Unknow". No nos piden ningún método especifico, asi que podremos usar el método que queramos.

In [23]:
# aunque ya lo hemos visto en la exploración inicial recordamos
print(f'La columna "state_province" tiene un {round((df2["state_province"].isnull().sum() / df2.shape[0]) * 100, 2)} % de nulos')

La columna "state_province" tiene un 89.11 % de nulos


In [24]:
# los reemplazamos por 'Unknown'
df2['state_province'].replace(np.nan, 'Unknown', inplace=True)
df2['state_province'].value_counts()[:1] #comprobamos que se ha cambiado correctamente

Unknown    2234
Name: state_province, dtype: int64

7. Ahora nuestros jefes nos piden que saquemos las coordenadas de las provincias donde están ubicadas las universidades. Para eso nos piden que usemos la librería de geopy que aprendimos el día del repaso, [aquí](https://pypi.org/project/geopy/) la documentación. Para desarrollar este ejercicio deberéis:
    - Sacar los valores únicos de la columna state_province.
    - Algunos de los valores que tenemos están con siglas, y deberéis reemplazarlos por lo siguiente:
        - NV: reemplazalo por Nevada
        - TX: reemplazalo por Texas
        - IN: reemplazalo por Indianapolis
        - CA: reemplazalo por California
        - VA: reemplazalo por Virginia
        - NY: reemplazalo por New York
        - MI: reemplazalo por Michigan
        - GA: reemplazalo por Georgia
        - ND: reemplazalo por North Dakota
    - Otros valores que tenemos más formateados son y que deberemos reemplazar:
        - New York, NY. Deberéis reemplazarlo por "New York".
        - 'Buenos Aires', 'Ciudad Autónoma de Buenos Aires'. En este caso deberéis poner en ambos casos "Buenos Aires"
    - Una vez realizados los pasos anteriores, crea una lista con los valores únicos de las provincias de las universidades.
    - Usando la API de geopy, extraed la latitud y la longitud de cada una de las provincias y almacenad los resultados en un dataframe.
    - Una vez que tengáis los datos del ejercicio anterior en un dataframe, unidlo con el de las universidades que hemos sacado de la API.

In [25]:
# obtenemos los valores únicos
df2['state_province'].unique()

array(['Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', 'Unknown',
       'Santiago Del Estero', 'Misiones', 'Catamarca', 'Formosa', 'Jujuy',
       'La Rioja', 'La Pampa', 'San Juan', 'San Luis', 'Tucumán',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Pennsylvania', 'NV', 'Iowa', 'VA', 'TX', 'Colorado', 'IN', 'CA',
       'South Carolina', 'Washington', 'NY', 'Texas', 'ND', 'MI', 'Ohio',
       'Florida', 'California', 'North Carolina', 'Michigan', 'GA',
       'New York, NY'], dtype=object)

In [26]:
# para reemplazar todos los valores a la vez utilizamos .replace() con un diccionario
df2['state_province'] = df2['state_province'].replace(spv.sustituir_provincias)
df2['state_province'].unique()

array(['Buenos Aires', 'Entre Ríos', 'Salta', 'Córdoba', 'Mendoza',
       'Santa Fé', 'Unknown', 'Santiago Del Estero', 'Misiones',
       'Catamarca', 'Formosa', 'Jujuy', 'La Rioja', 'La Pampa',
       'San Juan', 'San Luis', 'Tucumán', 'Quebec', 'Ontario',
       'Nova Scotia', 'British Columbia', 'Alberta', 'Manitoba',
       'New Brunswick', 'Saskatchewan', 'Newfoundland and Labrador',
       'Prince Edward Island', 'Yukon', 'Pennsylvania', 'Nevada', 'Iowa',
       'Virginia', 'Texas', 'Colorado', 'Indianapolis', 'California',
       'South Carolina', 'Washington', 'New York', 'North Dakota',
       'Michigan', 'Ohio', 'Florida', 'North Carolina', 'Georgia'],
      dtype=object)

In [27]:
# creamos la lista de las provincias con los valores únicos
provincias = df2['state_province'].unique().tolist()
provincias[:5] # printeamos una muestra como comprobación

['Buenos Aires', 'Entre Ríos', 'Salta', 'Córdoba', 'Mendoza']

In [28]:
print(f'Nº de provincias: {len(provincias)}')
print('Eliminamos el elemento "Unknown"')
provincias.remove('Unknown') # eliminamos este elemento para no solicitar sus coordenadas
print(f'Nº de provincias: {len(provincias)}')

Nº de provincias: 45
Eliminamos el elemento "Unknown"
Nº de provincias: 44


In [29]:
# usando la API de geopy obtenemos los datos
df_geopy = pd.DataFrame(columns=['state_province', 'latitude', 'longitude'])
for loc in provincias:
    try:
        geolocator = Nominatim(user_agent='Maitane') # inicializamos el geolocator
        location = geolocator.geocode(loc) # solicitamos la localización
        df = pd.DataFrame({'state_province': loc, 'latitude': location.latitude, 'longitude': location.longitude}, index=[0]) # obtenemos la latitud y longitud en un dataframe
        df_geopy = pd.concat([df_geopy, df], axis=0, ignore_index=True)
    except:
        print(f'La provincia {loc} no ha podido ser localizada')
df_geopy.head()

,state_province,latitude,longitude
0,Buenos Aires,-34.607568,-58.437089
1,Entre Ríos,-31.625284,-59.353958
2,Salta,-25.107670,-64.349496
3,Córdoba,37.884581,-4.776014
4,Mendoza,-34.787093,-68.438187


In [30]:
# recordamos el dataframe que teníamos
print(f'El dataframe general tiene {df2.shape[0]} filas y {df2.shape[1]} columnas')
print('Comprobamos que el indice es contínuo:')
df2.index

El dataframe general tiene 2507 filas y 5 columnas
Comprobamos que el indice es contínuo:


RangeIndex(start=0, stop=2507, step=1)

In [31]:
# y el dataframe de las coordenadas tiene las sigueintes características
print(f'El dataframe de las coordenadas tiene {df_geopy.shape[0]} filas y {df_geopy.shape[1]} columnas')
print('Comprobamos que el indice es contínuo:')
df_geopy.index

El dataframe de las coordenadas tiene 44 filas y 3 columnas
Comprobamos que el indice es contínuo:


RangeIndex(start=0, stop=44, step=1)

In [32]:
# unimos ambos dataframes
df2 = df2.merge(df_geopy, how='left', on='state_province').reset_index(drop=True)
df2.head()

,country,state_province,web_pages,name,alpha_two_code,latitude,longitude
0,Argentina,Buenos Aires,http://www.atlantida.edu.ar/,Universidad Atlantida Argentina,AR,-34.607568,-58.437089
1,Argentina,Buenos Aires,http://www.austral.edu.ar/,Universidad Austral Buenos Aires,AR,-34.607568,-58.437089
2,Argentina,Buenos Aires,http://www.caece.edu.ar/,"Universidad CAECE, Buenos Aires",AR,-34.607568,-58.437089
3,Argentina,Buenos Aires,http://www.cema.edu.ar/,Instituto Universitario CEMA,AR,-34.607568,-58.437089
4,Argentina,Buenos Aires,http://www.iese.edu.ar/,Instituto de Enseñanza Superior del Ejército,AR,-34.607568,-58.437089


In [33]:
print(f'El dataframe completo tiene {df2.shape[0]} filas y {df2.shape[1]} columnas')
print('Comprobamos que el indice es contínuo:')
df2.index

El dataframe completo tiene 2507 filas y 7 columnas
Comprobamos que el indice es contínuo:


RangeIndex(start=0, stop=2507, step=1)

In [34]:
# comprobamos que en los casos en los que la 'state_province' es 'Unknown' tenemos NaN en las columnas de latitid y longitud
df2[df2['state_province'] == 'Unknown'].sample(5)

,country,state_province,web_pages,name,alpha_two_code,latitude,longitude
2187,United States,Unknown,http://www.roguecc.edu,Rogue Community College,US,NaN,NaN
932,United States,Unknown,http://www.oakland.edu/,Oakland University,US,NaN,NaN
595,United States,Unknown,http://www.dillard.edu/,Dillard University,US,NaN,NaN
1533,United States,Unknown,http://www.blackrivertech.edu,Black River Technical College,US,NaN,NaN
449,United States,Unknown,http://www.brandeis.edu/,Brandeis University,US,NaN,NaN


In [35]:
# guardamos el dataframe limpio en .csv y .pkl
df2.to_csv('../datos/universidades_arg-can_usa_limpio.csv')
df2.to_pickle('../datos/universidades_arg-can_usa_limpio.pkl')

8. Crea una BBDD en mysql que contenga las siguientes tablas:  

    ![imagen](../datos/8-bbdd.png)  
    
    - Tabla países: donde encontraremos las siguientes columnas:
        - idestado: primary key, integer, autoincremental
        - nombre_pais: varchar
        - nombre_provincia: varchar
        - latitud: decimal
        - longitud: decimal
    - Tabla universidades: donde encontraremos las siguientes columnas:
        - iduniversidades: primary key, integer, autoincremental
        - nombre_universidad: varchar
        - pagina_web: varchar
        - paises_idestado: foreing key

In [36]:
# creamos la base de datos
mydb = mysql.connector.connect(
  host='localhost',
  user='root',
  password='AlumnaAdalab')

print('Conexión realizada con éxito')

mycursor = mydb.cursor()

try:
    mycursor.execute(f'CREATE DATABASE IF NOT EXISTS bd_universidades;')
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print('Error Code:', err.errno)
    print('SQLSTATE', err.sqlstate)
    print('Message', err.msg)

Conexión realizada con éxito
CMySQLCursor: CREATE DATABASE IF NOT EXISTS bd_univers..


In [37]:
# creamos la tabla de paises   
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                                    host='127.0.0.1', database='bd_universidades')

mycursor = cnx.cursor()

try: 
    mycursor.execute(spv.tabla_paises)
    cnx.commit() 

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

In [38]:
# creamos la tabla de universidades
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                                    host='127.0.0.1', database='bd_universidades')

mycursor = cnx.cursor()

try: 
    mycursor.execute(spv.tabla_universidades)
    cnx.commit() 

except mysql.connector.Error as err:
    print(err)
    print('Error Code:', err.errno)
    print('SQLSTATE', err.sqlstate)
    print('Message', err.msg)

9. Introduce todo el código que habéis ido creando en funciones, siguiendo la misma lógica que hemos seguido en los pairs

In [39]:
# A continuación repetimos el código completo como quedaría con las funciones para facilitar su lectura
# los paises que nos interesan son:
print(f'Vamos a realizar la extracción para los siguientes paises: {spv.paises}')
# utilizamos una función para obtener los datos de Argentina a modo de ejemplo para explorar los resultados que tendremos para cada país
argentina = spf.extraccion_json(spv.paises[0]) # devuelve el .json para explorarlo
print('Printeamos los dos primeros resultados descargados de la API a modo de ejemplo:')
argentina[0:2] # printeamos una muestra

Vamos a realizar la extracción para los siguientes paises: ['Argentina', 'Canada', 'United States']
Status code extracción Argentina: 200
Reason extracción Argentina: OK
El resultado obtenido es una lista de diccionarios. Nº de elementos de la lista resultado: 87
Las keys de cada diccionario son 6: dict_keys(['state-province', 'domains', 'name', 'country', 'web_pages', 'alpha_two_code'])
Printeamos los dos primeros resultados descargados de la API a modo de ejemplo:


[{'state-province': 'Buenos Aires',
  'domains': ['atlantida.edu.ar'],
  'name': 'Universidad Atlantida Argentina',
  'country': 'Argentina',
  'web_pages': ['http://www.atlantida.edu.ar/'],
  'alpha_two_code': 'AR'},
 {'state-province': 'Buenos Aires',
  'domains': ['austral.edu.ar'],
  'name': 'Universidad Austral Buenos Aires',
  'country': 'Argentina',
  'web_pages': ['http://www.austral.edu.ar/'],
  'alpha_two_code': 'AR'}]

In [40]:
# En este caso son solo tres paises pero por si fueran más podríamos automatizar la obtención de los datos, una vez conocida su estructura de la siguiente manera:
df_f = pd.DataFrame()
for pais in spv.paises:
    df_f = spf.extraccion_api_df(pais, df_f) # llamamos a la función de extracción que devuelve el dataframe
    print('-------------------------------------')
print(f'El dataframe resultado tiene {df_f.shape[0]} filas y {df_f.shape[1]} columnas')

Status code extracción Argentina: 200
Reason extracción Argentina: OK
Nº de elementos de la lista resultado de Argentina: 87
Las keys de cada diccionario de Argentina son 6: dict_keys(['state-province', 'domains', 'name', 'country', 'web_pages', 'alpha_two_code'])
Añadimos al dataframe resultado las 87 filas y 6 columnas de Argentina
-------------------------------------
Status code extracción Canada: 200
Reason extracción Canada: OK
Nº de elementos de la lista resultado de Canada: 154
Las keys de cada diccionario de Canada son 6: dict_keys(['state-province', 'domains', 'name', 'country', 'web_pages', 'alpha_two_code'])
Añadimos al dataframe resultado las 154 filas y 6 columnas de Canada
-------------------------------------
Status code extracción United States: 200
Reason extracción United States: OK
Nº de elementos de la lista resultado de United States: 2281
Las keys de cada diccionario de United States son 6: dict_keys(['state-province', 'domains', 'name', 'country', 'web_pages', '

In [41]:
# Antes de continuar realizamos una pequeña exploración del dataframe
spf.explorar_df(df_f, nombre = 'Argentina-Canada-USA')

EXPLORACIÓN DEL DATAFRAME ARGENTINA-CANADA-USA
---------------------------------------------------------------------------
Las primeras 5 filas del dataframe Argentina-Canada-USA son:


,state-province,domains,name,country,web_pages,alpha_two_code
0,Buenos Aires,[atlantida.edu.ar],Universidad Atlantida Argentina,Argentina,[http://www.atlantida.edu.ar/],AR
1,Buenos Aires,[austral.edu.ar],Universidad Austral Buenos Aires,Argentina,[http://www.austral.edu.ar/],AR
2,Ciudad Autónoma de Buenos Aires,[caece.edu.ar],"Universidad CAECE, Buenos Aires",Argentina,[http://www.caece.edu.ar/],AR
3,Ciudad Autónoma de Buenos Aires,[cema.edu.ar],Instituto Universitario CEMA,Argentina,[http://www.cema.edu.ar/],AR
4,Ciudad Autónoma de Buenos Aires,[iese.edu.ar],Instituto de Enseñanza Superior del Ejército,Argentina,[http://www.iese.edu.ar/],AR


---------------------------------------------------------------------------
Las últimas 5 filas del dataframe Argentina-Canada-USA son:


,state-province,domains,name,country,web_pages,alpha_two_code
2517,None,[vul.edu],Virginia University of Lynchburg,United States,[https://www.vul.edu/],US
2518,None,[voorhees.edu],Voorhees University,United States,[https://www.voorhees.edu/],US
2519,None,[wvstate.edu],West Virginia State University,United States,[https://www.wvstateu.edu/],US
2520,None,[wileyc.edu],Wiley College,United States,[https://www.wileyc.edu/],US
2521,None,[wssu.edu],Winston-Salem State University,United States,[https://www.wssu.edu/],US


---------------------------------------------------------------------------
A comntinuación se muestran 10 filas aleatorias del dataframe Argentina-Canada-USA:


,state-province,domains,name,country,web_pages,alpha_two_code
2022,None,[gccnj.edu],Gloucester County College,United States,[http://www.gccnj.edu],US
2457,None,[harrisburg.psu.edu],Pennsylvania State University - Harrisburg,United States,[https://harrisburg.psu.edu/],US
521,None,[cuny.edu],City University of New York,United States,[http://www.cuny.edu/],US
427,None,[benedictine.edu],Benedictine College,United States,[http://www.benedictine.edu/],US
162,British Columbia,[royalroads.ca],Royal Roads University,Canada,[http://www.royalroads.ca/],CA
2269,Texas,[schreiner.edu],Schreiner University,United States,[https://schreiner.edu/],US
232,Ontario,[yorku.ca],York University,Canada,[http://www.yorku.ca/],CA
868,None,[mwsc.edu],Missouri Western State College,United States,[http://www.mwsc.edu/],US
1776,None,[maccormac.edu],MacCormac College,United States,[http://www.maccormac.edu],US
1811,None,[allencc.edu],Allen County Community College,United States,[http://www.allencc.edu],US


---------------------------------------------------------------------------
El dataframe Argentina-Canada-USA tiene 2522 filas y 6 columnas
---------------------------------------------------------------------------
A continuación el resultado del método .info() incluyendo los tipos de dato de cada columna:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2522 entries, 0 to 2521
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   state-province  273 non-null    object
 1   domains         2522 non-null   object
 2   name            2522 non-null   object
 3   country         2522 non-null   object
 4   web_pages       2522 non-null   object
 5   alpha_two_code  2522 non-null   object
dtypes: object(6)
memory usage: 118.3+ KB
---------------------------------------------------------------------------
El número de nulos por columna en valor absoluto y porcentaje es:
state-province: nº de nulos: 2249. % de nulos: 89

,count,unique,top,freq
state-province,273,49,Ontario,46
domains,2522,2491,[ccc.edu],7
name,2522,2507,Southeastern Community College,2
country,2522,3,United States,2281
web_pages,2522,2519,[http://www.coastalpines.edu/],2
alpha_two_code,2522,3,US,2281


---------------------------------------------------------------------------
El dataframe Argentina-Canada-USA tiene las siguientes columnas: 
Index(['state-province', 'domains', 'name', 'country', 'web_pages',
       'alpha_two_code'],
      dtype='object')
---------------------------------------------------------------------------
El numero de valores distintos de cada columna es:
state-province: 49
domains: 2491
name: 2507
country: 3
Los valores únicos de la columna "country" son: ['Argentina' 'Canada' 'United States']
web_pages: 2519
alpha_two_code: 3
Los valores únicos de la columna "alpha_two_code" son: ['AR' 'CA' 'US']


In [42]:
# antes de continuar realizamos una copia del dataframe original para realizar la limpieza sobre él
df2_f = df_f.copy()
df2_f.sample()

,state-province,domains,name,country,web_pages,alpha_two_code
2197,None,[lanecc.edu],Lane Community College,United States,[http://www.Lanecc.edu],US


In [43]:
# realizamos la limpieza, incluyendo la obtención de la latitud y longitud para cada provincia
df2_f = spf.limpieza(df2_f)
df2_f.sample()

Columnas homogeneizadas y ordenadas correctamente
Columnas redundante "domains" eliminada.
Se han separado los datos de la columna "web_pages". El nuevo dataframe tiene 2535 filas y 5 columnas
Índice reseteado correctamente.
Si filtramos por la columna "name" tenemos 28 duplicados
Duplicados eliminados. El nuevo dataframe tiene 2507 filas y 5 columnas
La columna "state_province" tiene un 89.11 % de nulos. Procedemos a reemplazarlos por "Unknown".
Valores de la columna "state_province" homogeneizados
Latitud y longitud de las provincias obtenidas
Se han inluido la latitud y longitud en el dataframe original. El dataframe completo tiene 2507 filas y 7 columnas
A continuación se muestra su indice para comprobar que es contínuo: RangeIndex(start=0, stop=2507, step=1)


,country,state_province,web_pages,name,alpha_two_code,latitude,longitude
421,United States,Unknown,http://www.bcm.edu/,Baylor College of Medicine,US,NaN,NaN


In [44]:
# comprobamos que en los casos en los que la 'state_province' es 'Unknown' tenemos NaN en las columnas de latitid y longitud
df2_f[df2_f['state_province'] == 'Unknown'].sample(5)

,country,state_province,web_pages,name,alpha_two_code,latitude,longitude
1121,United States,Unknown,http://www.stephens.edu/,Stephens College,US,NaN,NaN
301,United States,Unknown,http://www.calbaptist.edu/,California Baptist University,US,NaN,NaN
2482,United States,Unknown,https://www.morris.edu/,Morris College,US,NaN,NaN
2372,United States,Unknown,http://www.gtc.edu,Gateway Technical College,US,NaN,NaN
355,United States,Unknown,http://www.moreheadstate.edu/,Morehead State University,US,NaN,NaN


In [45]:
# creamos la base de datos
spf.crear_bbdd(spv.acceso_sql, 'bd_universidades')

Conexión realizada con éxito
1007: Can't create database 'bd_universidades'; database exists
Error Code: 1007
SQLSTATE None
Message Can't create database 'bd_universidades'; database exists


In [46]:
# creamos la tabla de paises
spf.crear_insertar_tabla(spv.acceso_bbdd, spv.tabla_paises)

1050: Table 'paises' already exists
Error Code: 1050
SQLSTATE None
Message Table 'paises' already exists


In [47]:
# creamos la tabla de universidades
spf.crear_insertar_tabla(spv.acceso_bbdd, spv.tabla_universidades)

1050: Table 'universidades' already exists
Error Code: 1050
SQLSTATE None
Message Table 'universidades' already exists


10. BONUS  
- Introduce los datos en la BBDD de SQL.
    

In [48]:
df3 = df2.copy()
df3.head()

,country,state_province,web_pages,name,alpha_two_code,latitude,longitude
0,Argentina,Buenos Aires,http://www.atlantida.edu.ar/,Universidad Atlantida Argentina,AR,-34.607568,-58.437089
1,Argentina,Buenos Aires,http://www.austral.edu.ar/,Universidad Austral Buenos Aires,AR,-34.607568,-58.437089
2,Argentina,Buenos Aires,http://www.caece.edu.ar/,"Universidad CAECE, Buenos Aires",AR,-34.607568,-58.437089
3,Argentina,Buenos Aires,http://www.cema.edu.ar/,Instituto Universitario CEMA,AR,-34.607568,-58.437089
4,Argentina,Buenos Aires,http://www.iese.edu.ar/,Instituto de Enseñanza Superior del Ejército,AR,-34.607568,-58.437089


In [49]:
condiciones = [(df3['state_province'] == 'Unknown') & (df3['country'] == 'Argentina'), 
               (df3['state_province'] == 'Unknown') & (df3['country'] == 'Canada'), 
               (df3['state_province'] == 'Unknown') & (df3['country'] == 'United States'),
               df3['state_province'] != 'Unknown']

In [50]:
opciones = ['Unknown-Argentina', 'Unknown-Canada', 'Unknown-United States', df3['state_province']]

In [51]:
df3['state_province_sql'] = np.select(condiciones, opciones)
df3.sample(5)

,country,state_province,web_pages,name,alpha_two_code,latitude,longitude,state_province_sql
1433,United States,Unknown,http://www.wgu.edu/,Western Governors University,US,NaN,NaN,Unknown-United States
2431,United States,Unknown,https://chaminade.edu/,Chaminade University of Honolulu,US,NaN,NaN,Unknown-United States
2410,United States,Florida,https://www.gulfcoast.edu/,Gulf Coast State College,US,27.756767,-81.463983,Florida
1280,United States,Unknown,http://twin-cities.umn.edu/,University of Minnesota - Twin Cities,US,NaN,NaN,Unknown-United States
2322,United States,Unknown,http://www.nvcc.edu,Northern Virginia Community College,US,NaN,NaN,Unknown-United States


In [52]:
df3['latitude'].replace(np.nan, 0, inplace=True)
df3['longitude'].replace(np.nan, 0, inplace=True)


In [69]:
df3['longitude'].dtypes

dtype('float64')

In [68]:
df3['latitude'].dtypes

dtype('float64')

In [55]:
# comprobamos que la función de comprobar las provincias funciona
a = spf.check_provincias(spv.acceso_bbdd)
a

[]

In [56]:
for indice, fila in df3.iterrows():
    query_provincia = f"""
                    INSERT INTO paises (nombre_pais, nombre_provincia, latitud, longitud) 
                    VALUES ('{fila["country"]}', '{fila["state_province_sql"]}', '{fila["latitude"]}', '{fila["longitude"]}');
                    """
    provincias = spf.check_provincias(spv.acceso_bbdd)
    lista_prov = []
    for x in provincias:
        lista_prov.append(x[0])
    if len(provincias) == 0 or fila["state_province_sql"] not in lista_prov: 
        spf.crear_insertar_tabla(spv.acceso_bbdd, query_provincia)
    else:
        pass
        #print(f"{fila['state_province_sql']} ya esta en nuestra BBDD")    

Buenos Aires ya esta en nuestra BBDD
Buenos Aires ya esta en nuestra BBDD
Buenos Aires ya esta en nuestra BBDD
Buenos Aires ya esta en nuestra BBDD
Buenos Aires ya esta en nuestra BBDD
Buenos Aires ya esta en nuestra BBDD
Buenos Aires ya esta en nuestra BBDD
Buenos Aires ya esta en nuestra BBDD
Buenos Aires ya esta en nuestra BBDD
Buenos Aires ya esta en nuestra BBDD
Buenos Aires ya esta en nuestra BBDD
Buenos Aires ya esta en nuestra BBDD
Buenos Aires ya esta en nuestra BBDD
Buenos Aires ya esta en nuestra BBDD
Buenos Aires ya esta en nuestra BBDD
Entre Ríos ya esta en nuestra BBDD
Buenos Aires ya esta en nuestra BBDD
Buenos Aires ya esta en nuestra BBDD
Buenos Aires ya esta en nuestra BBDD
Buenos Aires ya esta en nuestra BBDD
Buenos Aires ya esta en nuestra BBDD
1265: Data truncated for column 'latitud' at row 1
Error Code: 1265
SQLSTATE None
Message Data truncated for column 'latitud' at row 1
Buenos Aires ya esta en nuestra BBDD
Buenos Aires ya esta en nuestra BBDD
Unknown-Argentin

- BONUS. Crea una clase con todo el código generado en esta evaluación.

In [57]:
# A continuación repetimos el código completo como quedaría con la clase para facilitar su lectura
# los paises que nos interesan son:
print(f'Vamos a realizar la extracción para los siguientes paises: {spv.paises}')

Vamos a realizar la extracción para los siguientes paises: ['Argentina', 'Canada', 'United States']


In [58]:
# instanciamos la clase para explorar los datos de argentina a modo de ejemplo
argent = spc.Universidades_hipolabs_api()
argent

In [59]:
arg_json = argent.extraccion_json(spv.paises[0])
print('Printeamos los dos primeros resultados descargados de la API a modo de ejemplo:')
arg_json[0:2]

Status code extracción Argentina: 200
Reason extracción Argentina: OK
El resultado obtenido es una lista de diccionarios. Nº de elementos de la lista resultado: 87
Las keys de cada diccionario son 6: dict_keys(['state-province', 'domains', 'name', 'country', 'web_pages', 'alpha_two_code'])
Printeamos los dos primeros resultados descargados de la API a modo de ejemplo:


[{'state-province': 'Buenos Aires',
  'domains': ['atlantida.edu.ar'],
  'name': 'Universidad Atlantida Argentina',
  'country': 'Argentina',
  'web_pages': ['http://www.atlantida.edu.ar/'],
  'alpha_two_code': 'AR'},
 {'state-province': 'Buenos Aires',
  'domains': ['austral.edu.ar'],
  'name': 'Universidad Austral Buenos Aires',
  'country': 'Argentina',
  'web_pages': ['http://www.austral.edu.ar/'],
  'alpha_two_code': 'AR'}]

In [60]:
# En este caso son solo tres paises pero por si fueran más podríamos automatizar la obtención de los datos, una vez conocida su estructura de la siguiente manera:
df_c = pd.DataFrame()
arg_can_usa = spc.Universidades_hipolabs_api() #creamos nuestra instancia para los tres paises
for pais in spv.paises:
    df_c = arg_can_usa.extraccion_api_df(pais, df_c) # llamamos a la función de extracción que devuelve el dataframe
    print('-------------------------------------')
print(f'El dataframe resultado tiene {df_c.shape[0]} filas y {df_c.shape[1]} columnas')

Status code extracción Argentina: 200
Reason extracción Argentina: OK
El resultado obtenido es una lista de diccionarios. Nº de elementos de la lista resultado: 87
Las keys de cada diccionario son 6: dict_keys(['state-province', 'domains', 'name', 'country', 'web_pages', 'alpha_two_code'])
Nº de elementos de la lista resultado de Argentina: 87
Las keys de cada diccionario de Argentina son 6: dict_keys(['state-province', 'domains', 'name', 'country', 'web_pages', 'alpha_two_code'])
Añadimos al dataframe resultado las 87 filas y 6 columnas de Argentina
-------------------------------------
Status code extracción Canada: 200
Reason extracción Canada: OK
El resultado obtenido es una lista de diccionarios. Nº de elementos de la lista resultado: 154
Las keys de cada diccionario son 6: dict_keys(['state-province', 'domains', 'name', 'country', 'web_pages', 'alpha_two_code'])
Nº de elementos de la lista resultado de Canada: 154
Las keys de cada diccionario de Canada son 6: dict_keys(['state-pr

In [61]:
# Antes de continuar realizamos una pequeña exploración del dataframe
arg_can_usa.explorar_df(df_c, nombre = 'Argentina-Canada-USA')

EXPLORACIÓN DEL DATAFRAME ARGENTINA-CANADA-USA
---------------------------------------------------------------------------
Las primeras 5 filas del dataframe Argentina-Canada-USA son:


,state-province,domains,name,country,web_pages,alpha_two_code
0,Buenos Aires,[atlantida.edu.ar],Universidad Atlantida Argentina,Argentina,[http://www.atlantida.edu.ar/],AR
1,Buenos Aires,[austral.edu.ar],Universidad Austral Buenos Aires,Argentina,[http://www.austral.edu.ar/],AR
2,Ciudad Autónoma de Buenos Aires,[caece.edu.ar],"Universidad CAECE, Buenos Aires",Argentina,[http://www.caece.edu.ar/],AR
3,Ciudad Autónoma de Buenos Aires,[cema.edu.ar],Instituto Universitario CEMA,Argentina,[http://www.cema.edu.ar/],AR
4,Ciudad Autónoma de Buenos Aires,[iese.edu.ar],Instituto de Enseñanza Superior del Ejército,Argentina,[http://www.iese.edu.ar/],AR


---------------------------------------------------------------------------
Las últimas 5 filas del dataframe Argentina-Canada-USA son:


,state-province,domains,name,country,web_pages,alpha_two_code
2517,None,[vul.edu],Virginia University of Lynchburg,United States,[https://www.vul.edu/],US
2518,None,[voorhees.edu],Voorhees University,United States,[https://www.voorhees.edu/],US
2519,None,[wvstate.edu],West Virginia State University,United States,[https://www.wvstateu.edu/],US
2520,None,[wileyc.edu],Wiley College,United States,[https://www.wileyc.edu/],US
2521,None,[wssu.edu],Winston-Salem State University,United States,[https://www.wssu.edu/],US


---------------------------------------------------------------------------
A comntinuación se muestran 10 filas aleatorias del dataframe Argentina-Canada-USA:


,state-province,domains,name,country,web_pages,alpha_two_code
1510,None,[sheltonstate.edu],Shelton State Community College,United States,[http://www.sheltonstate.edu],US
1897,None,[baycollege.edu],Bay de Noc Community College,United States,[http://www.baycollege.edu/],US
1519,None,[eac.edu],Eastern Arizona College,United States,[http://www.eac.edu],US
2394,None,[ntc.edu],Northcentral Technical College,United States,[http://www.ntc.edu],US
2442,None,[einsteinmed.edu],Einsteinmed,United States,[https://www.einsteinmed.edu/],US
1166,None,[tju.edu],Thomas Jefferson University,United States,[http://www.tju.edu/],US
1377,None,[uwosh.edu],University of Wisconsin - Oshkosh,United States,[http://www.uwosh.edu/],US
383,None,[andrews.edu],Andrews University,United States,[http://www.andrews.edu/],US
425,None,[beloit.edu],Beloit College,United States,[http://www.beloit.edu/],US
2308,None,[alamo.edu],St Philip's College,United States,[http://www.alamo.edu/spc/],US


---------------------------------------------------------------------------
El dataframe Argentina-Canada-USA tiene 2522 filas y 6 columnas
---------------------------------------------------------------------------
A continuación el resultado del método .info() incluyendo los tipos de dato de cada columna:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2522 entries, 0 to 2521
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   state-province  273 non-null    object
 1   domains         2522 non-null   object
 2   name            2522 non-null   object
 3   country         2522 non-null   object
 4   web_pages       2522 non-null   object
 5   alpha_two_code  2522 non-null   object
dtypes: object(6)
memory usage: 118.3+ KB
---------------------------------------------------------------------------
El número de nulos por columna en valor absoluto y porcentaje es:
state-province: nº de nulos: 2249. % de nulos: 89

,count,unique,top,freq
state-province,273,49,Ontario,46
domains,2522,2491,[ccc.edu],7
name,2522,2507,Southeastern Community College,2
country,2522,3,United States,2281
web_pages,2522,2519,[http://www.coastalpines.edu/],2
alpha_two_code,2522,3,US,2281


---------------------------------------------------------------------------
El dataframe Argentina-Canada-USA tiene las siguientes columnas: 
Index(['state-province', 'domains', 'name', 'country', 'web_pages',
       'alpha_two_code'],
      dtype='object')
---------------------------------------------------------------------------
El numero de valores distintos de cada columna es:
state-province: 49
domains: 2491
name: 2507
country: 3
Los valores únicos de la columna "country" son: ['Argentina' 'Canada' 'United States']
web_pages: 2519
alpha_two_code: 3
Los valores únicos de la columna "alpha_two_code" son: ['AR' 'CA' 'US']


In [62]:
# antes de continuar realizamos una copia del dataframe original para realizar la limpieza sobre él
df2_c = df_c.copy()
df2_c.sample()

,state-province,domains,name,country,web_pages,alpha_two_code
1186,None,"[usafa.af.mil, usafa.edu]",United States Air Force Academy,United States,"[http://www.usafa.af.mil/, http://www.usafa.edu/]",US


In [63]:
# realizamos la limpieza, incluyendo la obtención de la latitud y longitud para cada provincia
df2_c = arg_can_usa.limpieza(df2_c)
df2_c.sample()

Columnas homogeneizadas y ordenadas correctamente
Columnas redundante "domains" eliminada.
Se han separado los datos de la columna "web_pages". El nuevo dataframe tiene 2535 filas y 5 columnas
Índice reseteado correctamente.
Si filtramos por la columna "name" tenemos 28 duplicados
Duplicados eliminados. El nuevo dataframe tiene 2507 filas y 5 columnas
La columna "state_province" tiene un 89.11 % de nulos. Procedemos a reemplazarlos por "Unknown".
Valores de la columna "state_province" homogeneizados
Latitud y longitud de las provincias obtenidas
Se han inluido la latitud y longitud en el dataframe original. El dataframe completo tiene 2507 filas y 7 columnas
A continuación se muestra su indice para comprobar que es contínuo: RangeIndex(start=0, stop=2507, step=1)


,country,state_province,web_pages,name,alpha_two_code,latitude,longitude
1211,United States,Unknown,http://www.ucr.edu/,"University of California, Riverside",US,NaN,NaN


In [64]:
# comprobamos que en los casos en los que la 'state_province' es 'Unknown' tenemos NaN en las columnas de latitid y longitud
df2_c[df2_c['state_province'] == 'Unknown'].sample(5)

,country,state_province,web_pages,name,alpha_two_code,latitude,longitude
2433,United States,Unknown,https://www.ojaiusd.org/,Ojai Unified School District,US,NaN,NaN
1926,United States,Unknown,http://www.itascacc.edu/,Itasca Community College,US,NaN,NaN
371,United States,Unknown,http://www.acofi.edu/,Albertson College of Idaho,US,NaN,NaN
1430,United States,Unknown,http://westcoastuniversity.edu/,West Coast University,US,NaN,NaN
1340,United States,Unknown,http://www.usl.edu/,University of Southwestern Louisiana,US,NaN,NaN


In [65]:
# creamos la base de datos
arg_can_usa.crear_bbdd(spv.acceso_sql, 'bd_universidades')

Conexión realizada con éxito
1007: Can't create database 'bd_universidades'; database exists
Error Code: 1007
SQLSTATE None
Message Can't create database 'bd_universidades'; database exists


In [66]:
# creamos la tabla de paises
arg_can_usa.crear_insertar_tabla2(spv.acceso_bbdd, spv.tabla_paises)

1050: Table 'paises' already exists
Error Code: 1050
SQLSTATE None
Message Table 'paises' already exists


In [67]:
# creamos la tabla de universidades
arg_can_usa.crear_insertar_tabla2(spv.acceso_bbdd, spv.tabla_universidades)

1050: Table 'universidades' already exists
Error Code: 1050
SQLSTATE None
Message Table 'universidades' already exists
